# Claude - PDF Support
- docs: https://platform.claude.com/docs/en/build-with-claude/pdf-support
- Claude에서 'PDF Support'을 지원한다.
- PDF에서의 텍스트 추출, 차트 분석, 이미지 이해 능력을 갖추고 있다.
- use case로, 문서를 다른 정규화된 포맷으로 바꾸는 작업이 소개되어 있다.
- PDF input 제한으로, 32MB 크기, 100 pages 제한이 있으며, 패스워드가 걸리지 않아 있어야 한다.
- 'PDF support' 기능은 vision capabilities(시각 이해능력)에 의존하기 때문에 claude vision 모델들과 같은 limitation, consideration들을 갖는다.
    - 개인적인 의견으로, OCR 형태로 LLM 모델을 사용하는 듯하다. multi language 지원 되는지 확인 필요하며, 글씨체 또한 고려 필요하다. 
    - claude의 vision limitation, consideration을 살펴보자면,
        - vision 작업 API 역시, 이미지 100 per request 제한이 있고, 32MB per request 제한이 있다.
        - 성능 최적화를 위해 이미지 한 쪽이 1568 pixel 크기 이상이거나, 1600 토큰 이상이라면, aspect ratio를 유지한 채로 input image resize 작업을 제안한다.
        - 이미지 크기가 크면 별다른 성능 향상 없이, TTFT(Time-to-First Token) 지연 시간만 늘어난다.
        - 200 pixel 크기 이하의 경우 성능 저하가 있기 때문에 그 이하 낮추지 않도록 한다.
        - TTFT를 위해 1.15 Megapixels 크기 이하로 resizing 하는 것을 추천한다.
        - 이미지 토큰수 계산은 $tokens = (width \times height) / 750 $ 와 같이 계산할 수 있다.
        - JPEG, PNG, GIF, WebP 포맷을 지원하며
        - 인물 식별 작업에 제한 사항이 있으며, 회전되거나 200 pixel 이하의 작은 이미지에 정확도가 낮아지고 hallucination이 있을 수 있다.
        - 이미지 내의 정확한 위치나 layout을 인식하는 공간 추론 능력이 부족하다.
        - 개체 개수 세는 것이 어렵고, AI 생성 이미지 판별 용도로 사용할 수 없으며, 부적절한 내용에는 요청을 거절하게 되어 있다.
    - 한국어 OCR에 대한 내용은 없는 듯하다. 이 부분은 평가 내용 추가하는 것을 고려해야 할 듯하다.
- Document Processing Mode로 2 가지가 있는데
    - Converse Document Chat: PDF의 텍스트만 추출해서, 그 텍스트 바탕으로 대화를 할 수 있다.
    - Claude PDF Chat: PDF 페이지 이미지를 보고 분석하여 대화를 할 수 있다. API에서 citations flag가 필요하며, 없을 경우 텍스트 추출만 이루어진다.
- 작동 원리는
    - PDF에서 각 페이지를 이미지로 바꾼다.
    - 텍스트 또한 추출하여 페이지 이미지와 함께 입력으로 들어가고
    - 추출 텍스트와 페이미 이미지를 참고하여 답변을 생성한다.
    - 반복적 입력에 대해 최적화 위해 prompt caching 기능을 제공한다.
    ```JSON
    {
        "type": "document",
        "source": {
            "type": "base64",
            "media_type": "application/pdf",
            "data": $PDF_BASE64
        },
        {
            "cache_control": {
                "type": "ephemeral"
            }
        }
    }
    ```
    - batch api에서 또한 사용할 수 있다.
- 가격은
    - 텍스트 토큰에 대해 부가되고
    - 역시 페이지 이미지에 대해서도 요금이 부가된다.


In [1]:
from dotenv import load_dotenv
from anthropic import Anthropic
import os

load_dotenv()

CLAUDE_API_KEY = os.getenv("CLAUDE_API_KEY")

client = Anthropic(api_key=CLAUDE_API_KEY)

In [52]:

response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=64000,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "url",
                        "url": "https://arxiv.org/pdf/2411.13768"
                    },
                    "cache_control": {
                        "type": "ephemeral"
                    },
                    "citations": {
                        "enabled": True
                    }
                }
            ]
        },
        {
            "role": "user", 
            "content": [{
                "type": "text",
                "text": "Markdown형태로 변환하고, AI Engineer가 쉽게 이해할 수 있도록 번역해줘"
            }]
        }
    ],
    stream=True
)

events = []
for event in response:
    events.append(event)


In [53]:
text = ''
for event in events:
    if event.type == "content_block_delta" and event.delta.type == "text_delta":
        text += event.delta.text

print(text)

# LLM 에이전트의 평가 주도 개발 및 운영: 프로세스 모델과 참조 아키텍처

## 초록

대규모 언어 모델(LLM)은 명확하지 않은 목표를 추구하고 배포 후 적응할 수 있는 시스템인 LLM 에이전트의 등장을 가능하게 했습니다. 이러한 에이전트를 평가하는 것은 그들의 행동이 개방형이고, 확률적이며, 시간에 따른 시스템 수준의 상호작용에 의해 형성되기 때문에 어렵습니다. 고정된 벤치마크와 정적 테스트 스위트를 중심으로 구축된 전통적인 평가 방법은 창발적 행동을 포착하지 못하고 라이프사이클 전반에 걸친 지속적인 적응을 지원하지 못합니다.

보다 체계적인 접근을 위해, 우리는 학술 및 산업 평가 관행을 종합하는 다성적 문헌 검토(MLR)를 수행했습니다. 이 연구 결과는 평가를 최종 체크포인트가 아닌 지속적이고 관리하는 기능으로 포함시키는 두 가지 경험적으로 도출된 아티팩트인 프로세스 모델과 참조 아키텍처를 직접적으로 알려줍니다. 이들은 함께 오프라인(개발 시간) 및 온라인(런타임) 평가를 폐쇄 피드백 루프 내에서 통합하는 평가 주도 개발 및 운영(EDDOps) 접근 방식을 구성합니다. 평가 증거가 런타임 적응과 관리된 재개발 모두를 주도하게 함으로써, EDDOps는 변화하는 목표, 사용자 요구 및 거버넌스 제약과 일치하는 LLM 에이전트의 더 안전하고 추적 가능한 진화를 지원합니다.

**키워드**: 에이전트, 에이전틱, AgentOps, 대규모 언어 모델, 기반 모델, 평가, 운영, 프로세스 모델, 참조 아키텍처

## 1. 서론

대규모 언어 모델(LLM)의 최근 발전은 복잡한 작업을 자율적으로 수행할 수 있는 LLM 기반 에이전트(LLM 에이전트)의 등장으로 이어졌습니다(예: AI Scientist). 사전 정의된 입력과 구조화된 작업 실행에 의존하는 전통적인 AI/LLM 시스템과 달리, LLM 에이전트는 높은 수준의 명확하지 않은 사용자 목표를 동적으로 해석합니다. 그들은 맥락을 인식하고, 추론하고, 계획하고, 워크플로우를 실행하며, 종종 외부 도구, 지식

In [54]:
print(events[0])
print(events[-2])

RawMessageStartEvent(message=Message(id='msg_01919M9F1RXJUp2woBkCCgep', content=[], model='claude-sonnet-4-5-20250929', role='assistant', stop_reason=None, stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=88842, output_tokens=1, server_tool_use=None, service_tier='standard')), type='message_start')
RawMessageDeltaEvent(delta=Delta(stop_reason='end_turn', stop_sequence=None), type='message_delta', usage=MessageDeltaUsage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=88842, output_tokens=14102, server_tool_use=None))


- 따로 PDF 파일에 대해 전처리가 필요없기 때문에 편하다.
- citation flag는 messages.content 내부에서 설정할 수 있다.
- response.usage에서 'cache_read_input_tokens' 필드가 있는 것으로 보아, input 문서가 캐싱된 듯하다. 
- Openai에서는 내부에서 자동으로 캐싱 routing이 되고 커스텀이 불가하지만, anthropic에서는 message 단위로 cache 설정할 수 있어서 캐시 관련 커스텀 제어가 가능하다.
- response.content.ciataion 통해 어떤 내용에서 인용했는지 알 수 있어서, RAG 이용할 때 검증할 때 유용할 것 같다.
- 내부에서 PDF 전처리된 페이지 이미지, 텍스트가 어떻게 모델에 입력되는지 자세히 모르기 때문에 작업 검증은 필요하다.
    - 기존 모델 param을 그대로 입력하는 것으로 보아, PDF 입력을 위해 fine-tuning된 모델을 사용하는 것은 아니며
    - `"type": "document"` 명시함으로써, 그냥 전처리 코드만 작동되지 않을까 싶다.
    - 페이지 사이에 걸쳐있는 문장의 경우도 있는데, 텍스트가 페이지 이미지 별로 파싱해서 들어간다면, 불완전한 문장이 존재할 것이고, 페이지 텍스트들이 모두 merge되어 들어간다면, 완전한 문장이 입력될 듯하다.
    - DocumentBlockParam content 스펙을 확인해보면, `context`, `title` 옵션을 넣을 수 있는데, 아마 추출된 텍스트는 `context`에 들어가고, 추출된 이미지는 image input으로 들어갈 듯하다.
- Document mode는 아직 OpenAI SDK에서는 사용할 수 없는 듯하다. [참고](https://platform.claude.com/docs/en/api/openai-sdk#messages-array-fields)
- PDF Support를 이용하여 markdown으로 변환할 때, 이미지 content에 대한 내용을 output하지 않기 때문에 아쉬움은 있다. tool로 이미지 생성 API를 사용할 수 있게 하면 될 듯하지만, 가격이 좀 나올 수 있겠다.
- Openai에서도 File input 통한 'PDF content understanding'을 지원한다. 역시 비슷하게 텍스트와 페이지 이미지를 추출하여 모델에 입력되는 원리다. [참고](https://platform.openai.com/docs/guides/pdf-files)